In [11]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [12]:
df = pd.read_csv('./data.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7455 entries, 0 to 7454
Data columns (total 7 columns):
Unnamed: 0    7455 non-null int64
lat           7455 non-null float64
lng           7455 non-null float64
no_bath       7455 non-null int64
no_bed        7455 non-null int64
no_toilets    7455 non-null int64
price         7455 non-null int64
dtypes: float64(2), int64(5)
memory usage: 407.8 KB


In [14]:
df = df.drop("Unnamed: 0", axis=1)

In [15]:
data = df.values

In [16]:
data.shape

(7455, 6)

In [17]:
X = data[:, 0:5]
Y = data[:, 5]

In [18]:
def baseline_model():
    model = Sequential()
    model.add(Dense(50, input_dim=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [19]:
seed = 7
numpy.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [20]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: 21488111215702.22 (16144314592699.94) MSE


In [23]:
lekki = pd.DataFrame({ 'lng': [6.4698], 'lat': [3.5852], 'no_bed': [1], 'no_toilets': [1], 'no_bath': [1] })

In [24]:
lekki = lekki.values

In [28]:
estimator.fit(X, Y)

In [29]:
estimator.predict(lekki)

array(252843.265625, dtype=float32)

In [31]:
model_json = estimator.model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

estimator.model.save_weights("model.h5")

print("Saved model to disk")

Saved model to disk
